In [ ]:
%%capture
! pip install --upgrade mdai

import mdai
import json
domain = # your domain, e.g. public.md.ai or staging.md.ai
access_token = # your external access token
mdai_client = mdai.Client(domain= domain, access_token= access_token)

In [ ]:
# function to transform a markdown format file to txt plan text,
# pure text might be better for LLM to understand (no need to deal with weird
# markdown syntax)

import markdown
import re

def convert_markdown_to_text(markdown_file):
    with open(markdown_file, 'r') as file:
        markdown_text = file.read()

    # Convert Markdown to HTML
    html = markdown.markdown(markdown_text)

    # Remove HTML tags
    cleanr = re.compile('<.*?>')
    plain_text = re.sub(cleanr, '', html)

    # Remove Markdown syntax
    plain_text = re.sub(r'(\*\*|__)(.*?)\1', r'\2', plain_text)  # Remove bold text
    plain_text = re.sub(r'(\*|_)(.*?)\1', r'\2', plain_text)    # Remove italic text
    plain_text = re.sub(r'\[(.*?)\]\((.*?)\)', r'\1 (\2)', plain_text)  # Convert links to plain text format
    plain_text = re.sub(r'^\s*#(.*?)$', r'\1', plain_text, flags=re.MULTILINE)  # Remove headings

    # Remove leading/trailing whitespaces
    plain_text = plain_text.strip()
    return plain_text


In [ ]:
def gpt_response_generation(user_content_p1, system_content_p2, markdown_file):
  text_data = convert_markdown_to_text(markdown_file)
  knowledge_base = "You can extract the answer from here: but be careful the code that involves == is replaced with = =, so if you want to show code as your answer, please make sure you present as == not = =, also \
  remember that a lot of the knowledge back involve backslash n because it's the code interpretion, you should think of it just as to the next line \
  and backslash quotation mark is just quotation mark, if the question is about code, try give as much as you can, if it is not asking for code, you could keep your conceptual response simple and straightforward, \
  but do not forget to make your response on point, make sure there is no extra code or comment that goes beyond what the question asks" + text_data
  user_content_p2 = user_content_p1 + "\n" + knowledge_base

  # user_content_p2 = user_content_p2.replace("```python", "```txt")
  # user_content_p2 = user_content_p2.replace("`", " ")
  # user_content_p2 = user_content_p2.replace(" == ", " = = ")
  # user_content_p2 = user_content_p2.replace("/.bashrc", "/. bashrc")
  # user_content_p2 = user_content_p2.replace("&lt;", "<")
  # user_content_p2 = user_content_p2.replace("&gt;", ">")
  # user_content_p2 = user_content_p2[:]

  messages_p2 = [{"role": "system", "content": system_content_p2},
            {"role": "user", "content": user_content_p2}]
  # messages_p2 = [{"role": "system", "content": system_content_p2},
  #           {"role": "user", "content": ""}]
  result = mdai_client.chat_completion.create(messages_p2, model='gpt-3.5-turbo', temperature=0)
  return result["choices"][0]["message"]["content"]


In [ ]:
user_content_p1 = "What are the three files that I could download to install MD.ai Command line interface"
system_content_p2 = "You are a bot helping with answering a question according to a documentation page of a medicial AI website."
markdown_file = '/content/installation.md'
result = gpt_response_generation(user_content_p1, system_content_p2, markdown_file)
result

What are the three files that I could download to install MD.ai Command line interface
You can extract the answer from here: but be careful the code that involves = = is replaced with = =, so if you want to show code as your answer, please make sure you present as = = not = =, also   remember that a lot of the knowledge back involve backslash n because it's the code interpretion, you should think of it just as to the next line   and backslash quotation mark is just quotation mark, if the question is about code, try give as much as you can, if it is not asking for code, you could keep your conceptual response simple and straightforward,   but do not forget to make your response on point, make sure there is no extra code or comment that goes beyond what the question asksInstalling the MD.ai Command-line Interface (CLI)
The MD.ai CLI is a convenient tool for efficiently accessing a subset of the MD.ai API, such as creating new projects and datasets, uploading large files to project datase

'To install the MD.ai Command-line Interface (CLI), you need to download three files:\n\n1. For Windows: Download the zip file from this link: [Windows CLI](https://storage.googleapis.com/mdai-app-data/cli-releases/mdaicli{{ LATESTVERSION }}windows_amd64.zip)\n\n2. For MacOS: Download the zip file from this link: [MacOS CLI](https://storage.googleapis.com/mdai-app-data/cli-releases/mdaicli{{ LATESTVERSION }}darwin_amd64.zip)\n\n3. For Linux: Download the zip file from this link: [Linux CLI](https://storage.googleapis.com/mdai-app-data/cli-releases/mdaicli{{ LATESTVERSION }}linux_amd64.zip)\n\nAfter downloading the appropriate zip file for your platform, unzip the package. To run the `mdai` binary from any directory, make sure that the binary is available on the PATH. You can follow the instructions provided in the documentation for setting the PATH on your specific operating system:\n\n- [PATH instructions for Windows](https://stackoverflow.com/questions/1618280/where-can-i-set-path-to